In [ ]:
### Reduce road network to centerlines ###
### First, rasterize the road subset, e.g. in QGIS ###

from skimage.morphology import skeletonize
from skimage.util import invert
import gdal

# The road raster file (.tif)
datapath = ''

data = gdal.Open(datapath)
driver = gdal.GetDriverByName('GTiff')
x_size = data.RasterXSize  
y_size = data.RasterYSize 
geotrans = data.GetGeoTransform()  
proj = data.GetProjection() 

band = data.GetRasterBand(1)
# Set the output path here
bandname = ''
band_arr = band.ReadAsArray()
band_arr = band_arr.astype(int)
# Do this if other values than 0 and 1 are present in the image. The background (all non-road classes) should be 0.
band_arr[band_arr > 1] = 1

# Invert the image
image = invert(band_arr)

# perform skeletonization
skeleton = skeletonize(band_arr)

band_arr = band.ReadAsArray()
dataset = driver.Create(bandname, x_size, y_size, 1, gdal.GDT_Float32)
dataset.GetRasterBand(1).SetNoDataValue(float('nan'))
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.GetRasterBand(1).WriteArray(skeleton)
dataset.FlushCache()
dataset = None